# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Shiguai,40.7058,110.2856,47.30,40,29,5.19,CN,2023-10-30 21:14:31
1,1,Puerto Natales,-51.7236,-72.4875,44.80,49,96,5.26,CL,2023-10-30 21:14:32
2,2,Maintirano,-18.0667,44.0167,78.08,73,8,13.96,MG,2023-10-30 21:09:32
3,3,Cascais,38.6979,-9.4215,62.06,80,100,11.99,PT,2023-10-30 21:12:14
4,4,Cabo San Lucas,22.8909,-109.9124,89.62,68,40,9.22,MX,2023-10-30 21:12:00


In [3]:
#checking for data types within the dataframe
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
                                      "Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.4,
                                      size = "Humidity",
                                      tiles = "OSM",
                                      frame_width = 700,
                                      frame_height = 500     
                                     )


# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

'''
A max temperature lower than 70 degrees but higher than 85
Wind speed less than 4.2 m/s
Cloudiness less than 7

'''

weather_criteria = city_data_df.loc[(city_data_df["Max Temp"] > 60)&
                                    (city_data_df["Max Temp"] < 85)& 
                                    (city_data_df["Wind Speed"] < 4.2)& 
                                    (city_data_df["Cloudiness"] < 7)]

# Drop any rows with null values
weather_criteria.dropna()

# Display sample data
weather_criteria

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
153,153,Gariaband,20.6350,82.0614,62.69,82,2,1.68,IN,2023-10-30 21:14:57
248,248,Nishinoomote,30.7333,131.0000,69.44,58,0,1.61,JP,2023-10-30 21:15:22
265,265,Grand-Lahou,5.1367,-5.0242,77.74,91,1,3.38,CI,2023-10-30 21:15:25
294,294,Namibe,-15.1961,12.1522,65.86,81,0,4.18,AO,2023-10-30 21:15:30
308,308,Ghat,24.9647,10.1728,77.63,18,0,4.12,LY,2023-10-30 21:15:32
354,354,Mach,29.8637,67.3298,65.93,50,5,1.79,PK,2023-10-30 21:15:39
415,415,Argostolion,38.1811,20.4890,68.74,58,0,2.59,GR,2023-10-30 21:15:47
445,445,North Bend,43.4065,-124.2243,66.11,42,0,0.00,US,2023-10-30 21:13:25
495,495,Dadhar,29.4753,67.6514,78.57,29,3,1.10,PK,2023-10-30 21:15:59
527,527,Zakynthos,37.7914,20.8953,66.63,66,0,3.94,GR,2023-10-30 21:16:06


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_criteria[['City', 'Country', 'Lat','Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
153,Gariaband,IN,20.6350,82.0614,82,
248,Nishinoomote,JP,30.7333,131.0000,58,
265,Grand-Lahou,CI,5.1367,-5.0242,91,
294,Namibe,AO,-15.1961,12.1522,81,
308,Ghat,LY,24.9647,10.1728,18,
354,Mach,PK,29.8637,67.3298,50,
415,Argostolion,GR,38.1811,20.4890,58,
445,North Bend,US,43.4065,-124.2243,42,
495,Dadhar,PK,29.4753,67.6514,29,
527,Zakynthos,GR,37.7914,20.8953,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    
    "categories": "accommodation.hotel","apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Gariaband - nearest hotel: No hotel found
Nishinoomote - nearest hotel: ゼウスハウス
Grand-Lahou - nearest hotel: Cap Lahou
Namibe - nearest hotel: Hotel Chik Chik Namibe
Ghat - nearest hotel: فندق تاسيلي
Mach - nearest hotel: No hotel found
Argostolion - nearest hotel: Ainos Hotel
North Bend - nearest hotel: The Mill Casino Hotel
Dadhar - nearest hotel: No hotel found
Zakynthos - nearest hotel: Phoenix Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
153,Gariaband,IN,20.6350,82.0614,82,No hotel found
248,Nishinoomote,JP,30.7333,131.0000,58,ゼウスハウス
265,Grand-Lahou,CI,5.1367,-5.0242,91,Cap Lahou
294,Namibe,AO,-15.1961,12.1522,81,Hotel Chik Chik Namibe
308,Ghat,LY,24.9647,10.1728,18,فندق تاسيلي
354,Mach,PK,29.8637,67.3298,50,No hotel found
415,Argostolion,GR,38.1811,20.4890,58,Ainos Hotel
445,North Bend,US,43.4065,-124.2243,42,The Mill Casino Hotel
495,Dadhar,PK,29.4753,67.6514,29,No hotel found
527,Zakynthos,GR,37.7914,20.8953,66,Phoenix Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
hotel_map_locate = hotel_df.hvplot.points(
                                      "Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.4,
                                      size = "Humidity",
                                      tiles = "OSM",
                                      frame_width = 600,
                                      frame_height = 400,     
                                      hover_cols = ["Hotel Name", "Country"]
                                     )

# Display the map
hotel_map_locate

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)